# Run streamlit app from a Google Colab Notebook
> Adapted from https://medium.com/@jcharistech/how-to-run-streamlit-apps-from-colab-29b969a1bdfc

In [ ]:
!pip install -q streamlit
!pip install -q pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.1/238.1 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 KB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 KB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# Authenticate the NGROK
!pip install -q pyngrok
!ngrok authtoken 2IYm3NhniVMUdjkxMZDIyzpRAqG_4E9dB8DetNRd9yWv3vt12

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Reset the execution environment after streamlit installation

## Create file called app.py

In [ ]:
# Install the SQLAlchemy library if it is not installed
!sudo apt-get install python3-dev libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
!sudo pip3 install -U sql_magic > /dev/null
!pip install psycopg2-binary > /dev/null

In [ ]:
!pip install streamlit-folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.3/102.3 KB 10.5 MB/s eta 0:00:00
  Attempting uninstall: folium
    Found existing installation: folium 0.12.1.post1
    Uninstalling folium-0.12.1.post1:
      Successfully uninstalled folium-0.12.1.post1


In [ ]:
!pip install PyPDF2
!python -q -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 6.0 MB/s eta 0:00:00
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-11 03:46:02.498163: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
%%writefile app.py
import streamlit as st
from streamlit_folium import st_folium
import pandas as pd
from cachetools import cached
import matplotlib.pyplot as plt
import numpy as np
import PyPDF2

from sqlalchemy import create_engine
from map import *
import en_core_web_md 
nlp = en_core_web_md.load()

# Connect to the MySQL, but without selecting a database

#see appendix!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

#storing entire df to refer back to after filtering
df = pd.read_sql('SELECT * FROM jobdata', con=engine) # Original dataframe
st.session_state['original'] = df

#temporary dataframe for filtered results, will start with entire df on intialization
if 'df' not in st.session_state:
  st.session_state['df'] = df
 

PAGE_CONFIG = {"page_title":"More Examples","page_icon":":smiley:","layout":"centered"}
st.set_page_config(**PAGE_CONFIG)

#running the page
def main():

    menu = ["Home","Map View","by State", "by Company"]
    choice = st.sidebar.selectbox('Menu',menu)

    #default
    if choice == 'Home':
      
      #reset session state df
      st.session_state['df'] = st.session_state['original'] 

      #Title and subheader
      st.subheader("Home")
      st.title("Job Matches")

      #user input for job names
      job = st.text_input('Search For Jobs')
      st.write("Keyword: " + job)
      
      #selecting job category
      #cat_select = st.selectbox('Job Categories', df['cat'].unique())
      #st.write(cat_select)

      #resume upload section
      resume = st.file_uploader("Upload your resume in PDF format", ['pdf'])

      run = st.button('Run Job Matching')
      
      #scanning resume for text
      if resume != None and job != None and run:
        df = st.session_state['df']
        #scanning resume for text
        pdfReader = PyPDF2.PdfFileReader(resume)
        pageObj = pdfReader.getPage(0) 
        resume_str = pageObj.extractText()

        #filtering dataframe for titles that contain string job
        #encoding filtered entries using nlp
        df['title'] = df['title'].str.lower()
        returns = df[df['title'].str.contains(job)]
        encoded_desc = returns['desc'].apply(nlp)
        
        #encoding resume using nlp to draw similarity score
        encoded_resume = nlp(resume_str)
        resume_sim = [encoded_resume.similarity(x) for x in encoded_desc.tolist()]

        #appending resume_sim series to dataframe then sorting by similiarty score
        #returning first 100 entries after sorting
        returns['similarity score'] = resume_sim
        st.session_state["df"] = returns.sort_values(by=['similarity score'], ascending = False)


        #displaying top 50 (by similiarty score) matches as dataframe
        st.write(st.session_state["df"].sort_values(by=['similarity score'], ascending = False).head(50)[['title', 'cat', 'State', 'similarity score']])

        # Restart Search
        # ADD CODE TO DETECT RESET FUNCTIONALITY
        # st.session_state["df"] = st.session_state["original_df"]
  
    elif choice == 'Map View':
      loadMap(st.session_state["df"])
        
    elif choice == 'by State':
      plotValCnt('State', st.session_state["df"], True)
    
    elif choice == 'by Company':
      plotValCnt('comp', st.session_state["df"], False)

  

#running main        
if __name__ == '__main__':
    main()

Writing app.py


In [ ]:
%%writefile map.py
import math
import folium
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from streamlit_folium import st_folium
import streamlit as st

#wrapper function for either bar or pie type charts, True for chart_type will yield pie chart, False yields bar chart
def plotValCnt(col, dataframe, chart_type):
  nums = []
  names = []
  series = dataframe[col].value_counts()
  valCntDF = pd.DataFrame(series)
  # print(valCntDF.columns)
  for i, row in valCntDF.iterrows():
    names.append(i)
    nums.append(row[0])

  # valCntDF = pd.DataFrame({'count': nums},
  #                   index=names)
  # print(series)
  # plot = valCntDF.plot.pie(y='count', figsize=(10, 10))
  
  if(chart_type):
    myPie(nums, names, col)
  
  else:
    myBar(nums, names, col)



def myPie(nums, names, col):
    #pie chart:
  # numbers from https://stackoverflow.com/questions/72338356/how-to-show-values-in-pandas-pie-chart
  # labels from https://matplotlib.org/stable/gallery/pie_and_polar_charts/pie_and_donut_labels.html
  
  fig, ax = plt.subplots(figsize=(32, 8), subplot_kw=dict(aspect="equal"))
  wedges, texts, autotexts = ax.pie(nums, autopct=lambda pct: str(int(pct/100.*np.sum(nums))), textprops=dict(color="w"))

  bbox_props = dict(boxstyle="square,pad=0.3", fc="w", ec="k", lw=0.72)
  kw = dict(arrowprops=dict(arrowstyle="-"),
            bbox=bbox_props, zorder=0, va="center")
  for i, p in enumerate(wedges):
      ang = (p.theta2 - p.theta1)/2. + p.theta1
      y = np.sin(np.deg2rad(ang))
      x = np.cos(np.deg2rad(ang))
      horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
      connectionstyle = "angle,angleA=0,angleB={}".format(ang)
      kw["arrowprops"].update({"connectionstyle": connectionstyle})
      ax.annotate(names[i], xy=(x, y), xytext=(1.35*np.sign(x), 1.4*y),
                  horizontalalignment=horizontalalignment, **kw)

  # ax.legend(wedges, names,
  #           title="States",
  #           loc="center left",
  #           bbox_to_anchor=(1, 0, 0.5, 1))
  plt.setp(autotexts, size=8, weight="light") #weight could be bold
  ax.set_title("Job Postings Per " + col)
  st.pyplot(plt)

def myBar(nums, names, col):
  #https://www.geeksforgeeks.org/bar-plot-in-matplotlib/

  # Figure Size
  fig, ax = plt.subplots(figsize =(16, 9))
  # Horizontal Bar Plot
  ax.barh(names, nums)

  # Remove axes splines
  for s in ['top', 'bottom', 'left', 'right']:
      ax.spines[s].set_visible(False)
  # Remove x, y Ticks
  ax.xaxis.set_ticks_position('none')
  ax.yaxis.set_ticks_position('none')
  
  # Show top values
  ax.invert_yaxis()
  
  # Add annotation to bars
  for i in ax.patches:
      plt.text(i.get_width()+0.2, i.get_y()+0.5,
              str(round((i.get_width()), 2)),
              fontsize = 8, fontweight ='bold',
              color ='grey')
  
  # Add Plot Title
  ax.set_title('Job Postings Per ' + col,
              loc ='left', )
  
  # Show Plot
  st.pyplot(plt)

def loadMap(filtered_df):
  
  fmap = folium.Map(location=[40.73, -95], zoom_start=5,  tiles='cartodbpositron')

  nonGraphed = []

  for name, row in filtered_df.iterrows():
      # if row["lon"] > -40:
      #   print(row)

      opacity = .25
      color = "green" if row["comp"] == 'Siemens' else ("pink" if row["comp"] == 'Lyft' else ("blue" if row["comp"] == 'AMR' else ("orange" if row["comp"] == "Amazon Delivery Service Partners" else "yellow")))

      size = 10
      popupHeight = 300
      html = """<p style='font-family:sans-serif;font-size:11px'>"""
      for lbl, val in row.iteritems():
        if (lbl == "redURL"):
          html += "<a href="+val+">redirect url</a> <br>"
        elif not (lbl =="index" or lbl == "desc"):
          html += "<strong>"+lbl+": </strong>" + str(val) + "<br>"

      popup = folium.Popup(folium.IFrame(html=html, width=300, height=popupHeight), max_width=200)

      # We create a marker on the map and we add it to the map
      if not math.isnan(row["lat"]):
        folium.CircleMarker(location=[row["lat"], row["lon"]], 
                            radius = size,
                            color='black', weight=0.5, 
                            popup = popup,
                            fill=True,
                            fill_opacity = opacity,
                            fill_color = color,
                          ).add_to(fmap)
      else:
        nonGraphed.append(html)

  st_data = st_folium(fmap, width = 500)



Writing map.py


In [ ]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
!streamlit run app.py &>/dev/null &
public_url = ngrok.connect(8501)
public_url


#finance, sales, software

<NgrokTunnel: "http://701a-34-73-17-178.ngrok.io" -> "http://localhost:8501">

In [ ]:
input("press enter to stop the streamlit app")
# This is to stop the tunnel and streamlit
ngrok.kill()
!pkill streamlit

In [ ]:
!streamlit run app.py &>/dev/null &

In [ ]:
from pyngrok import ngrok
# Setup a tunnel to the streamlit port 8501
public_url = ngrok.connect(8501)
public_url

In [ ]:
# If ngrok isn't working use this.
!npm install localtunnel
!npx localtunnel --port 8501